<a href="https://colab.research.google.com/github/RubanZ/ppa-for-da/blob/master/students/%D0%A0%D1%83%D0%B1%D0%B0%D0%BD_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80/%D0%9E%D1%82%D0%B1%D0%BE%D1%80_%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Отбор признаков на примере датасета об сердечно-сосудистых заболеваниях

Датасет: https://www.kaggle.com/rishidamarla/heart-disease-prediction




#Введение
Данные могут содержать большой объем признаков. И не всегда весь объем необъодим для выполнения задачи. Критерий отбора признаков включает релевантность для целей анализа данных, качество и технические ограничения, такие как объем данных или тип данных.

Удаление избыточных признаков позволяет лучше понять данные, а также сократить время настройки модели, улучшить её точность и облегчить интерпретируемость.

Сильно коррелированные друг с другом переменные дают модели одну и ту же информацию, следовательно, для анализа не нужно использовать их все. Если использовать, то модель окажется переобучена (overfit) и предвзята относительно одного отдельного признака.



###*РЕВЬЮ*
*Опечатка - "необъодим"*

#Методы фильтрации

Установка библиотек

In [1]:
pip install numpy pandas matplotlib sklearn

Импорт библиотек и данных

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE, f_classif
from sklearn.ensemble import RandomForestClassifier
df = pd.read_csv("Heart_Disease_Prediction.csv")

print('Размер Dataframe:', df.shape)
print(df.head(5))

FileNotFoundError: ignored

In [ ]:
print('\n\nИнформация о Dataframe df.info():')
print(df.info())

In [ ]:
label = df["Heart Disease"]
df.drop("Heart Disease", axis=1, inplace=True)
print('\n\nЗначение метки "Сердечное заболевание":')
print(label.value_counts())

In [ ]:
label.value_counts().plot(kind="bar")

Изменение типов данных для признаков, имеющих категориальный характер. Однако, при печати столбцов ("Sex", "Chest pain type", "FBS over 120", "EKG results", "Exercise angina", "Slope of ST", "Number of vessels fluro", "Thallium") мы наблюдаем, что они определяются, как данные целого типа и рассматриваются как целые числа

In [ ]:
categorical_features = ["Sex", "Chest pain type", "FBS over 120", "EKG results", "Exercise angina", "Slope of ST", "Number of vessels fluro", "Thallium"]
df[categorical_features] = df[categorical_features].astype("category")

Теперь будем масштабировать наши непрерывные функции с помощью MinMaxScaler. Это тип нормализации, когда значения загоняются в диапазоне от 0 до 1, согласно уравнению (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

In [ ]:
continuous_features = set(df.columns) - set(categorical_features)
scaler = MinMaxScaler()
df_norm = df.copy()
df_norm[list(continuous_features)] = scaler.fit_transform(df[list(continuous_features)])

##Отбор признаков с использованием рекурсивного исключения признаков (RFE)
RFE является широко используемой техникой/алгоритмом для выбора точного числа значимых признаков. Иногда метод используется, чтобы объяснить некоторое число «самых важных» признаков, влияющих на результаты; а иногда для уменьшения очень большого числа переменных (около 200-400), и оставляются только те, которые вносят хоть какой-то вклад в модель, а все остальные исключаются. RFE использует ранговую систему. Признакам в наборе данных выставляются ранги. Затем эти ранги используются для рекурсивного исключения признаков в зависимости от коллинеарности между ними и значимости этих признаков в модели.

In [ ]:
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=5)
rfe_norm = df_norm
X_new = rfe.fit_transform(rfe_norm, label)
X_new

In [ ]:
clf = RandomForestClassifier()
clf.fit(rfe_norm, label)
# create a figure to plot a bar, where x axis is features, and Y indicating the importance of each feature
plt.figure(figsize=(12,12))
plt.bar(rfe_norm.columns, clf.feature_importances_)
plt.xticks(rotation=45)

Гистограмма показывает важность каждого признака (атрибута, измерения, набора данных). В нашем случае Thallium и Number of vessels fluro являются наиболее важными характеристиками, но большинство из них имеют важное значение, и в этом случае в значительной степени стоит передать эти признаки нашей модели машинного обучения.

#Отбор по методу X^2
Отбор признков с помощью хи-квадрат для каждого неотрицательного объекта и класса. Эта оценка может использоваться для выбора функций n_features с наивысшими значениями для тестовой статистики хи-квадрат из X, которая должна содержать только неотрицательные функции, такие как логические значения или частоты (например, количество терминов в классификации документа), относительно классы. Использование этой функции «отсеивает» признаки, которые с наибольшей вероятностью будут независимыми от класса и, следовательно, не имеют отношения к классификации.

In [ ]:
x2_norm = df_norm
X_new = SelectKBest(k=5, score_func=chi2).fit_transform(x2_norm, label)
X_new

In [ ]:
clf = RandomForestClassifier()
clf.fit(x2_norm, label)
# Вывод
plt.figure(figsize=(12,12))
plt.bar(x2_norm.columns, clf.feature_importances_)
plt.xticks(rotation=45)

В этом случае Chest pain type и Thllium являются наиболее важными характеристиками.

#ANOVA
Дисперсионный анализ (ANOVA) может определить, различаются ли средние значения трех или более групп. ANOVA использует F-тесты для статистической проверки равенства средних.

In [ ]:
anova_norm = df_norm
X_new = SelectKBest(k=5, score_func=f_classif).fit_transform(anova_norm, label)
X_new

In [ ]:
clf = RandomForestClassifier()
clf.fit(anova_norm, label)
# Вывод
plt.figure(figsize=(12,12))
plt.bar(anova_norm.columns, clf.feature_importances_)
plt.xticks(rotation=45)

В этом случае Chest pain type и Thllium являются наиболее важными характеристиками.

#Источники:

https://habr.com/ru/post/550978/

https://tproger.ru/translations/feature-engineering-in-machine-learning/

http://www.machinelearning.ru/wiki/index.php?title=CRISP-DM/Data_Preparation

https://waksoft.susu.ru/2021/02/09/otbor-priznakov-s-pomoshhyu-scikit-learn-v-python/

https://en.wikipedia.org/wiki/Chi-squared_distribution

https://blog.minitab.com/en/adventures-in-statistics-2/understanding-analysis-of-variance-anova-and-the-f-test

#Ревью. Итоги.

Минусы:
*   Не запустилось "по клику", нужно самому загружать датасет

Плюсы:
*  Можно выделить логическую структуру
*  Есть список источников
*  Есть ссылка на датасет

Итог: Хороший туториал, можно добавить автоматическую загрузку и распаковку датасета. Можно дать больше ссылок внутри туториала на каждый конкретный метод.

